In [ ]:
import google.generativeai as genai
import pygame

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
%%time
response = model.generate_content("What is the largest city in France?")